<a href="https://colab.research.google.com/github/Deelaw15/Movie-Recommendation-System/blob/main/Matrix_Factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2555156 sha256=cd79b40d3c428d4caea65afe8b308ec2744a771ead9dd8b1d65597f905e8ff9f
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [5]:
!pip install 'numpy<2'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 57.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy

In [1]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
import pandas as pd
import os

In [2]:
!git clone https://github.com/Deelaw15/Movie-Recommendation-System.git

Cloning into 'Movie-Recommendation-System'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 73 (delta 33), reused 41 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (73/73), 8.15 MiB | 11.67 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [3]:
os.chdir('/content/Movie-Recommendation-System/ml-latest-small')

In [5]:
!ls

links.csv  movies.csv  ratings.csv  README.txt	tags.csv


In [7]:
train = pd.read_csv("ratings.csv")
train.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
# Load your train data (from earlier step)
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)


In [9]:
# Split into train/test for evaluation
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [10]:
# Initialize SVD model
svd_model = SVD(n_factors=100, biased=True, random_state=42)

In [11]:
# Train model
svd_model.fit(trainset)

In [12]:
# Evaluate
from surprise import accuracy
predictions = svd_model.test(testset)
rmse = accuracy.rmse(predictions)
print(f"✅ SVD RMSE: {rmse:.4f}")

RMSE: 0.8807
✅ SVD RMSE: 0.8807


Fine tuning

In [13]:
from surprise.model_selection import GridSearchCV

In [14]:
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 40, 60],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.1, 0.2]
}

grid = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1)
grid.fit(data)

In [16]:
print("✅ Best RMSE:", grid.best_score['rmse'])
print("🏆 Best Parameters:", grid.best_params['rmse'])

✅ Best RMSE: 0.857427952929192
🏆 Best Parameters: {'n_factors': 150, 'n_epochs': 60, 'lr_all': 0.01, 'reg_all': 0.1}
